In [2]:
library("DESeq2")

### RNA-seq: raw counts

In [4]:
count_data <- read.delim("../data/210706_rawcounts.csv.gz", row.names = 1, sep = ",")
head(count_data,3)
dim(count_data)

,T0063,T0096,T0103,T0249,T0294,T0305,T0315,T0342,T0430,T0644,⋯,T6985,T6987,T6990,T6991,T6993,T6995,T6999,T7001,T7002,T7003
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000000003,2881,1061,1406,1057,2628,714,1320,2094,1651,1404,⋯,592,707,509,492,669,825,442,280,1029,962
ENSG00000000005,2,0,0,27,4,5,6,54,9,1,⋯,0,1,0,1,10,1,0,6,3,6
ENSG00000000419,412,538,473,628,527,377,728,635,811,280,⋯,536,727,359,492,607,816,406,307,747,731


[1] 58735   244

### Normalization

In [5]:
meta <- data.frame(row.names = colnames(count_data))
meta["sample"] <-1
dds <- DESeqDataSetFromMatrix(countData = count_data,colData = meta, design = ~ 1)

In [6]:
# uses the "median ratio method" described in Anders and Huber (2010)
dds <- estimateSizeFactors(dds) 
norm_counts <-  counts(dds, normalized=TRUE)
head(norm_counts,3)

,T0063,T0096,T0103,T0249,T0294,T0305,T0315,T0342,T0430,T0644,⋯,T6985,T6987,T6990,T6991,T6993,T6995,T6999,T7001,T7002,T7003
ENSG00000000003,2794.910890,1037.4077,1323.1914,891.8690,2312.492048,660.667226,1233.510248,1844.00880,1351.152283,1638.030191,⋯,523.1762,631.1342941,537.9177,467.3740779,837.57266,753.8453175,478.7253,372.198057,892.625710,900.766646
ENSG00000000005,1.940237,0.0000,0.0000,22.7819,3.519775,4.626521,5.606865,47.55324,7.365458,1.166688,⋯,0.0000,0.8926935,0.0000,0.9499473,12.51977,0.9137519,0.0000,7.975673,2.602407,5.618087
ENSG00000000419,399.688749,526.0371,445.1419,529.8900,463.730331,348.839698,680.299591,559.19083,663.709571,326.672688,⋯,473.6866,648.9881638,379.3958,467.3740779,759.95009,745.6215504,439.7341,408.088584,647.999422,684.470289


### Filtering 
* keep genes with 15+ norm. counts
* in at least 10 samples

In [7]:
filtering <- function(count_data) {
  non_zero <- rowSums(count_data >= 15)
  passed_genes <- which(non_zero >= 10)
  passed_genes_names <- rownames(count_data)[passed_genes]
  cat("passed genes:", length(passed_genes_names), "\n")
  count_data <- count_data[passed_genes_names, ]
  return(count_data)
}

norm_filtered <- filtering(norm_counts)
rownames(norm_filtered) <- make.names(rownames(norm_filtered), unique = TRUE)
norm_filtered <- norm_filtered[order(rownames(norm_filtered)), ]
head(norm_filtered, 3)

passed genes: 23758 


,T0063,T0096,T0103,T0249,T0294,T0305,T0315,T0342,T0430,T0644,⋯,T6985,T6987,T6990,T6991,T6993,T6995,T6999,T7001,T7002,T7003
ENSG00000000003,2794.910890,1037.4077,1323.1914,891.8690,2312.492048,660.667226,1233.510248,1844.00880,1351.152283,1638.030191,⋯,523.1762,631.1342941,537.9177,467.3740779,837.57266,753.8453175,478.7253,372.198057,892.625710,900.766646
ENSG00000000005,1.940237,0.0000,0.0000,22.7819,3.519775,4.626521,5.606865,47.55324,7.365458,1.166688,⋯,0.0000,0.8926935,0.0000,0.9499473,12.51977,0.9137519,0.0000,7.975673,2.602407,5.618087
ENSG00000000419,399.688749,526.0371,445.1419,529.8900,463.730331,348.839698,680.299591,559.19083,663.709571,326.672688,⋯,473.6866,648.9881638,379.3958,467.3740779,759.95009,745.6215504,439.7341,408.088584,647.999422,684.470289


In [9]:
log_norm_filtered <- log2(norm_filtered+1)
# uncomment to save file
#write.table(log_norm_filtered, "../data/UKE244_log2exprs.tsv.gz", sep = "\t", row.names = TRUE, col.names = NA, quote = FALSE)
head(log_norm_filtered, 3)

,T0063,T0096,T0103,T0249,T0294,T0305,T0315,T0342,T0430,T0644,⋯,T6985,T6987,T6990,T6991,T6993,T6995,T6999,T7001,T7002,T7003
ENSG00000000003,11.449103,10.020157,10.370896,9.802305,11.175856,9.369962,10.269723,10.849412,10.401042,10.678627,⋯,9.033908,9.3040873,9.073921,8.8715174,9.711792,9.5600372,8.906065,8.543798,9.803527,9.816610
ENSG00000000005,1.555932,0.000000,0.000000,4.571792,2.176251,2.492243,2.723966,5.601496,3.064444,1.115492,⋯,0.000000,0.9204408,0.000000,0.9634351,3.756999,0.9364038,0.000000,3.166020,1.848961,2.726414
ENSG00000000419,8.646338,9.041761,8.801359,9.052269,8.860250,8.450550,9.412146,9.129775,9.376580,8.356112,⋯,8.890831,9.3442696,8.571358,8.8715174,9.571658,9.5442333,8.783765,8.676269,9.342073,9.420950
